In [3]:
"""
map_data_clean.ipynb

File used to clean up the map/census data from 
https://chronicdata.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-Place-Data-202/eav7-hnsx/data

This dataset will be used to create map visualizations
"""

'\nmap_data_clean.ipynb\n\nFile used to clean up the map/census data from \nhttps://chronicdata.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-Place-Data-202/eav7-hnsx/data\n\nThis dataset will be used to create map visualizations\n'

In [27]:
# Importing statements

import pandas as pd
import math

In [28]:
# Importing raw dataset

df = pd.read_csv('PLACES.csv')

In [29]:
# Preview dataset

print(df.columns)
df.head()

Index(['Year', 'StateAbbr', 'StateDesc', 'LocationName', 'DataSource',
       'Category', 'Measure', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value', 'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit', 'TotalPop18plus',
       'TotalPopulation', 'Geolocation', 'LocationID', 'CategoryID',
       'MeasureId', 'DataValueTypeID', 'Short_Question_Text'],
      dtype='object')


,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,...,Low_Confidence_Limit,High_Confidence_Limit,TotalPop18plus,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text
0,2022,TX,Texas,Sandy Point,BRFSS,Health-Related Social Needs,Received food stamps in the past 12 months amo...,%,Crude prevalence,9.5,...,8.4,10.6,158,207,POINT (-95.4817319 29.4012556),4865345,SOCLNEED,FOODSTAMP,CrdPrv,Food Stamps
1,2022,TX,Texas,Sandy Point,BRFSS,Health-Related Social Needs,Received food stamps in the past 12 months amo...,%,Age-adjusted prevalence,10.2,...,9.1,11.4,158,207,POINT (-95.4817319 29.4012556),4865345,SOCLNEED,FOODSTAMP,AgeAdjPrv,Food Stamps
2,2022,TX,Texas,Sandy Point,BRFSS,Health Status,Fair or poor self-rated health status among ad...,%,Crude prevalence,21.8,...,19.8,23.8,158,207,POINT (-95.4817319 29.4012556),4865345,HLTHSTAT,GHLTH,CrdPrv,General Health
3,2022,TX,Texas,Sandy Point,BRFSS,Health Status,Fair or poor self-rated health status among ad...,%,Age-adjusted prevalence,20.9,...,18.9,22.8,158,207,POINT (-95.4817319 29.4012556),4865345,HLTHSTAT,GHLTH,AgeAdjPrv,General Health
4,2022,TX,Texas,Sandy Point,BRFSS,Disability,Hearing disability among adults,%,Crude prevalence,7.4,...,6.6,8.2,158,207,POINT (-95.4817319 29.4012556),4865345,DISABLT,HEARING,CrdPrv,Hearing Disability


In [7]:
# Column values exploration

print('Unique Data_Value_Unit: ', df.Data_Value_Unit.unique(), '\n')
print('Unique StateDesc: ', df.StateDesc.unique(), '\n')
print('Unique Year: ', df.Year.unique(), '\n')
print('Unique Short_Question_Text: ', df.Short_Question_Text.unique(), '\n')
print('Unique Measure: ', df.Measure.unique(), '\n')

Unique Data_Value_Unit:  ['%'] 

Unique StateDesc:  ['Texas' 'Utah' 'Vermont' 'Virginia' 'Washington' 'West Virginia'
 'Wisconsin' 'Wyoming' 'Alabama' 'Alaska' 'Arizona' 'Arkansas'
 'California' 'Colorado' 'Connecticut' 'Delaware' 'Florida'
 'District of Columbia' 'Georgia' 'Hawaii' 'Idaho' 'Illinois' 'Indiana'
 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland' 'Massachusetts'
 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana' 'Nebraska'
 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Rhode Island' 'South Carolina' 'South Dakota' 'Tennessee'] 

Unique Year:  [2022 2021] 

Unique Short_Question_Text:  ['Food Stamps' 'General Health' 'Hearing Disability' 'High Cholesterol'
 'Housing Insecurity' 'Independent Living Disability' 'Social Isolation'
 'Transportation Barriers' 'Physical Inactivity' 'Mammography'
 'Frequent Mental Distress' 'Mobility Disability' 'Obesity'
 'Frequent Physi

In [30]:
# How many from 2020 vs 2019

print(len(df[df['Year'] == 2020]))
print(len(df[df['Year'] == 2019]))

0
0


In [31]:
# Splitting up dataset by years

df_2020 = df[df['Year'] == 2020]
df_2019 = df[df['Year'] == 2019]

In [32]:
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

# Ensure 'StateAbbr' exists in the dataset before proceeding
if 'StateAbbr' in df_2020.columns and 'StateAbbr' in df_2019.columns:
    df_2020['FIP'] = df_2020['StateAbbr'].map(state_codes)
    df_2019['FIP'] = df_2019['StateAbbr'].map(state_codes)
else:
    print("❌ Column 'StateAbbr' is missing from the dataset!")


In [33]:
df_2020.head()

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,...,High_Confidence_Limit,TotalPop18plus,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text,FIP


In [34]:
# Removing unnecessary columns from datasets, keeping only the ones that exist
desired_cols = ['Year', 'StateDesc', 'LocationName', 'Measure', 'Category', 'Data_Value',
                'TotalPopulation', 'Geolocation', 'Short_Question_Text', 'FIP', 'StateAbbr']

# Filter columns based on actual dataset columns
existing_cols_2020 = [col for col in desired_cols if col in df_2020.columns]
existing_cols_2019 = [col for col in desired_cols if col in df_2019.columns]

df_2020 = df_2020[existing_cols_2020]
df_2019 = df_2019[existing_cols_2019]

df_2020.head()


,Year,StateDesc,LocationName,Measure,Category,Data_Value,TotalPopulation,Geolocation,Short_Question_Text,FIP,StateAbbr


In [35]:
# Removing NaN from Data_Value and TotalPopulatioin columns

df_2020=df_2020.dropna(subset=['Data_Value','TotalPopulation'])

In [37]:
import numpy as np

# Ensure 'Data_Value' and 'TotalPopulation' are numeric
df_2020['Data_Value'] = pd.to_numeric(df_2020['Data_Value'], errors='coerce')
df_2020['TotalPopulation'] = pd.to_numeric(df_2020['TotalPopulation'], errors='coerce')

# Drop rows with missing values before calculation
df_2020.dropna(subset=['Data_Value', 'TotalPopulation'], inplace=True)

# Calculate TotalNumber (rounding up)
df_2020['TotalNumber'] = np.ceil((df_2020['Data_Value'] / 100) * df_2020['TotalPopulation']).astype(int)

# Verify column creation
print(df_2020[['TotalNumber']].head())


Empty DataFrame
Columns: [TotalNumber]
Index: []


In [38]:
df_2020.head()

,Year,StateDesc,LocationName,Measure,Category,Data_Value,TotalPopulation,Geolocation,Short_Question_Text,FIP,StateAbbr,TotalNumber


In [39]:
import numpy as np

# Ensure 'Data_Value' and 'TotalPopulation' are numeric
df_2020['Data_Value'] = pd.to_numeric(df_2020['Data_Value'], errors='coerce')
df_2020['TotalPopulation'] = pd.to_numeric(df_2020['TotalPopulation'], errors='coerce')

# Drop rows with missing values before calculation
df_2020.dropna(subset=['Data_Value', 'TotalPopulation'], inplace=True)

# Calculate TotalNumber (rounding up)
df_2020['TotalNumber'] = np.ceil((df_2020['Data_Value'] / 100) * df_2020['TotalPopulation']).astype(int)

if 'TotalNumber' in df_2020.columns:
    sum_2020 = df_2020.groupby(['StateDesc', 'Short_Question_Text', 'Measure', 'StateAbbr'])['TotalNumber'].sum().reset_index()
    print(sum_2020.head())
else:
    print("❌ Column 'TotalNumber' is missing!")



Empty DataFrame
Columns: [StateDesc, Short_Question_Text, Measure, StateAbbr, TotalNumber]
Index: []


In [40]:
sum_2020.head()

,StateDesc,Short_Question_Text,Measure,StateAbbr,TotalNumber


In [41]:
# pulls the longitude from the Geolocation string
def pull_lon(x):
    str1 = x.split('(', 1)[1]
    str1 = str1.replace(')', '')
    return float(str1.split(' ', 1)[0])

# pulls the latitude from the Geolocation string
def pull_lat(x):
    str1 = x.split('(', 1)[1]
    str1 = str1.replace(')', '')
    return float(str1.split(' ', 1)[1])

In [43]:
import pandas as pd

# Function to extract latitude and longitude from '(lat, lon)' format
def pull_lat(geolocation):
    try:
        return float(geolocation.strip("()").split(",")[0]) if pd.notna(geolocation) else None
    except:
        return None

def pull_lon(geolocation):
    try:
        return float(geolocation.strip("()").split(",")[1]) if pd.notna(geolocation) else None
    except:
        return None

# Ensure 'Geolocation' column exists and has valid data
df_2020.dropna(subset=['Geolocation'], inplace=True)

# Apply functions to create latitude and longitude columns
df_2020['Latitude'] = df_2020['Geolocation'].apply(pull_lat)
df_2020['Longitude'] = df_2020['Geolocation'].apply(pull_lon)

# Verify the columns were created correctly
print(df_2020[['Geolocation', 'Latitude', 'Longitude']].head())


Empty DataFrame
Columns: [Geolocation, Latitude, Longitude]
Index: []


In [44]:
df_2020.head()

,Year,StateDesc,LocationName,Measure,Category,Data_Value,TotalPopulation,Geolocation,Short_Question_Text,FIP,StateAbbr,TotalNumber,Latitude,Longitude


In [45]:
popdf = pd.read_csv('populations.csv')

popdf.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,NPOPCHG_2020,NPOPCHG_2021,...,NETMIG2020,NETMIG2021,RESIDUAL2020,RESIDUAL2021,RBIRTH2021,RDEATH2021,RNATURALINC2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021
0,10,0,0,0,United States,331449281,331501080,331893745,51799,392665,...,12247,244622,0,0,10.798957,10.352637,0.446319,0.737485,0.000000,0.737485
1,20,1,0,0,Northeast Region,57609148,57525633,57159838,-83515,-365795,...,-79429,-333592,-1024,-1151,9.950554,10.492070,-0.541516,0.977386,-6.794897,-5.817511
2,20,2,0,0,Midwest Region,68985454,68935174,68841444,-50280,-93730,...,-47266,-85268,-718,5156,10.860566,11.058248,-0.197682,0.549222,-1.786994,-1.237772
3,20,3,0,0,South Region,126266107,126409007,127225329,142900,816322,...,134454,770076,528,-3385,11.168835,10.777476,0.391359,0.886268,5.186064,6.072332
4,20,4,0,0,West Region,78588572,78631266,78667134,42694,35868,...,4488,-106594,1214,-620,10.767153,8.947910,1.819243,0.487570,-1.842880,-1.355309


In [46]:
popdf = popdf[popdf['STATE'] != 0]
popdf = popdf[['NAME', 'ESTIMATESBASE2020']]



popdf.rename(columns={'NAME': 'StateDesc'}, inplace=True)

popdf.head()

,StateDesc,ESTIMATESBASE2020
5,Alabama,5024279
6,Alaska,733391
7,Arizona,7151502
8,Arkansas,3011524
9,California,39538223


In [47]:
df_cd = pd.merge(sum_2020, popdf, how='outer', on='StateDesc')

In [48]:
df_cd.head()

,StateDesc,Short_Question_Text,Measure,StateAbbr,TotalNumber,ESTIMATESBASE2020
0,Alabama,NaN,NaN,NaN,NaN,5024279
1,Alaska,NaN,NaN,NaN,NaN,733391
2,Arizona,NaN,NaN,NaN,NaN,7151502
3,Arkansas,NaN,NaN,NaN,NaN,3011524
4,California,NaN,NaN,NaN,NaN,39538223


In [49]:
# Saving the sum_2020 to a csv

df_cd.to_csv('grouped_2020.csv')

In [50]:
# Saving the df_2020 to a csv

df_2020.to_csv('measures_2020.csv')